In [1]:
import pandas as pd
import numpy as np
import matplotlib as  plt

%matplotlib inline
pd.set_option('display.max_columns', 50)
from urllib.request import urlopen
from bs4  import BeautifulSoup

import re
import requests


baseurl  = 'https://grocers.com.pk/categories'

# header for preventing the mod_security error
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
}

r = requests.get(url=baseurl,headers=headers)
soup = BeautifulSoup(r.content,'lxml')


In [2]:
soup

<!DOCTYPE html>
<html lang="en">
<head><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<!--[if IE]>
<meta http-equiv="X-UA-Compatible" content="IE=edge">
<![endif]-->
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<title>Categories List | Grocers</title>
<!-- Favicons Icon -->
<link href="https://grocers.com.pk/public//assets_frontend/fav.png" rel="icon" type="image/x-icon"/>
<link href="https://grocers.com.pk/public//assets_frontend/fav.png" rel="shortcut icon" type="image/x-icon"/>
<!-- Mobile Specific -->
<meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>
<!-- CSS Style -->
<link href="https://grocers.com.pk/public/assets_frontend/css/animate.css" rel="stylesheet" type="text/css"/>
<link href="https://grocers.com.pk/public/assets_frontend/css/bootstrap.min.css" rel="stylesheet" type="text/css"/>
<link href="https://grocers.com.pk/public/assets_frontend/css/style.css" rel="stylesheet" type="text/css"/>


In [3]:
allCatElements = soup.findAll('section', {"class":["col-main", "col-sm-12"]})

In [4]:
portfolio = []

for x in allCatElements:
    categoryElement = x.select('div.category-title > h1 > a')
    categoryTitle = categoryElement[0].getText()
    categoryLink = categoryElement[0]['href']
    subCategoriesElements = x.select('li.level3 > a')
    subCategoriesObjList = [{"link": x['href'], "subCategory": x.find('span').getText()} for x in subCategoriesElements]
    portfolio.append({
        "category": categoryTitle,
        "categoryLink": categoryLink,
        "subCategories": subCategoriesObjList
    })

portfolio

[{'category': 'Foodgrains, Oil & Masala',
  'categoryLink': 'https://grocers.com.pk/category/foodgrains,_oil_&_masala',
  'subCategories': [{'link': 'https://grocers.com.pk/category/atta,_flours_&_sooji',
    'subCategory': 'Atta, Flours & Sooji'},
   {'link': 'https://grocers.com.pk/category/rice', 'subCategory': 'Rice'},
   {'link': 'https://grocers.com.pk/category/daals_&_pulses',
    'subCategory': 'Daals & Pulses'},
   {'link': 'https://grocers.com.pk/category/salt,_sugar_&_gurr',
    'subCategory': 'Salt, Sugar & Gurr'},
   {'link': 'https://grocers.com.pk/category/edible_oils_&_ghee',
    'subCategory': 'Edible Oils & Ghee'},
   {'link': 'https://grocers.com.pk/category/recipe_masala',
    'subCategory': 'Recipe Masala'},
   {'link': 'https://grocers.com.pk/category/spices_&_ingredients',
    'subCategory': 'Spices & Ingredients'},
   {'link': 'https://grocers.com.pk/category/olives_%28green/black/sliced%29',
    'subCategory': 'OLIVES (Green/Black/Sliced)'}]},
 {'category': 'Br

In [5]:
def sendRequest(link, pNum):
    headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
    }
    cusLink = f"{link}?page={pNum}"
    r = requests.get(url=cusLink,headers=headers)
    soup = BeautifulSoup(r.content,'lxml')
    return soup


In [6]:
testLink = "https://grocers.com.pk/category/breakfast_&_dairy?page=1"

f = sendRequest(testLink)
alls = f.select('div.col-item')

alls[0].select('div.info > div > div.item-title > a')[0].getText() # gives out the name

alls[0].select('div.price-box > p > span.price ')[0] # [0] because taking the current price. Old price at [1]

alls


TypeError: sendRequest() missing 1 required positional argument: 'pNum'

In [7]:
def extractData(link, category, subCategory, pNum):
    data = []
    f = sendRequest(link, pNum)
    alls = f.select('div.col-item')
    for x in alls:
        name = x.select('div.info > div > div.item-title > a')[0].getText() # gives out the name
        price = int(x.select('div.price-box > p > span.price ')[0].getText().strip().split(" ")[1]) # [0] because taking the current price. Old price at [1]
        cat = category
        subCat = subCategory
        dObj = {
            "name": name,
            "price": price,
            "category": cat,
            "subCategory": subCat
        }
        data.append(dObj)
    return data
        
        

In [8]:

d = []
index = 1
for x in portfolio:
    for y in x['subCategories']:
        for z in range(0,10):
            pageNumber = z +1
            f = extractData(y['link'], x['category'], y["subCategory"], z) 
            if f:
                index += 1
                print(f"{index}: {f}")
                d.extend(f)
            else:
                continue
                
d

                

2: [{'name': 'Maida 500 grams', 'price': 48, 'category': 'Foodgrains, Oil & Masala', 'subCategory': 'Atta, Flours & Sooji'}, {'name': 'Sooji Semolina - Suji Powder - 500 grams', 'price': 39, 'category': 'Foodgrains, Oil & Masala', 'subCategory': 'Atta, Flours & Sooji'}, {'name': 'Baisan - Besan 500 grams', 'price': 95, 'category': 'Foodgrains, Oil & Masala', 'subCategory': 'Atta, Flours & Sooji'}, {'name': 'Mughal Chakki Atta Flour - 5 Kg', 'price': 370, 'category': 'Foodgrains, Oil & Masala', 'subCategory': 'Atta, Flours & Sooji'}, {'name': 'Sunridge Chakki Atta Flour - 5kg', 'price': 475, 'category': 'Foodgrains, Oil & Masala', 'subCategory': 'Atta, Flours & Sooji'}, {'name': 'Sunridge Chakki Atta Flour - 10kg', 'price': 824, 'category': 'Foodgrains, Oil & Masala', 'subCategory': 'Atta, Flours & Sooji'}, {'name': 'Ashrafi Fine Flour - Atta Poly Bag - 5kg', 'price': 337, 'category': 'Foodgrains, Oil & Masala', 'subCategory': 'Atta, Flours & Sooji'}, {'name': 'Sadi Chakki Atta Flour - 

8: [{'name': 'Al Barakah - Red Lobia 500 grams', 'price': 140, 'category': 'Foodgrains, Oil & Masala', 'subCategory': 'Daals & Pulses'}, {'name': 'Al Barakah - Daal Masoor - Dal lal 500 grams', 'price': 84, 'category': 'Foodgrains, Oil & Masala', 'subCategory': 'Daals & Pulses'}, {'name': 'Al Barakah - Daal Chana - Channa Dal 1 Kg', 'price': 205, 'category': 'Foodgrains, Oil & Masala', 'subCategory': 'Daals & Pulses'}, {'name': 'Al Barakah - Daal Mong - Yellow Dal Moong 500 grams', 'price': 172, 'category': 'Foodgrains, Oil & Masala', 'subCategory': 'Daals & Pulses'}, {'name': 'Al Barakah - Daal Malka Masoor kali Dal 1 kg', 'price': 149, 'category': 'Foodgrains, Oil & Masala', 'subCategory': 'Daals & Pulses'}, {'name': 'Al Barakah - Black Chana - Kaala Channa 500 grams', 'price': 86, 'category': 'Foodgrains, Oil & Masala', 'subCategory': 'Daals & Pulses'}, {'name': 'Al Barakah - Bajra 1kg', 'price': 73, 'category': 'Foodgrains, Oil & Masala', 'subCategory': 'Daals & Pulses'}, {'name': 

14: [{'name': 'Soya Supreme Cooking Oil 1Ltr', 'price': 262, 'category': 'Foodgrains, Oil & Masala', 'subCategory': 'Edible Oils & Ghee'}, {'name': 'Dalda Cooking Oil Fortified 1 Ltr Pouch', 'price': 270, 'category': 'Foodgrains, Oil & Masala', 'subCategory': 'Edible Oils & Ghee'}, {'name': 'Mezan Canola Oil Pouch 1 Ltr', 'price': 297, 'category': 'Foodgrains, Oil & Masala', 'subCategory': 'Edible Oils & Ghee'}, {'name': 'Dalda Ghee VTF Banaspati Pouch 1kg', 'price': 272, 'category': 'Foodgrains, Oil & Masala', 'subCategory': 'Edible Oils & Ghee'}, {'name': 'Mezan Banaspati Pouch 1kg', 'price': 265, 'category': 'Foodgrains, Oil & Masala', 'subCategory': 'Edible Oils & Ghee'}, {'name': 'Tullo Cooking Oil 1L', 'price': 265, 'category': 'Foodgrains, Oil & Masala', 'subCategory': 'Edible Oils & Ghee'}, {'name': 'Sufi Soyabean Oil Bottles 3 Ltr', 'price': 976, 'category': 'Foodgrains, Oil & Masala', 'subCategory': 'Edible Oils & Ghee'}, {'name': 'Sufi Sunflower Oil Bottle 1.8 Ltr', 'price':

20: [{'name': 'Borges Extra Virgin Olive Oil 2L', 'price': 3302, 'category': 'Foodgrains, Oil & Masala', 'subCategory': 'Edible Oils & Ghee'}, {'name': 'Rafhan Corn Oil Tin 10 Liter', 'price': 3859, 'category': 'Foodgrains, Oil & Masala', 'subCategory': 'Edible Oils & Ghee'}, {'name': 'Sasso Extra Virgin Olive Oil Tin 3Ltr', 'price': 4881, 'category': 'Foodgrains, Oil & Masala', 'subCategory': 'Edible Oils & Ghee'}, {'name': 'Mezan Cooking Oil Pouch 1 Ltr', 'price': 259, 'category': 'Foodgrains, Oil & Masala', 'subCategory': 'Edible Oils & Ghee'}, {'name': 'Rafhan Corn Oil 5 Liter', 'price': 2092, 'category': 'Foodgrains, Oil & Masala', 'subCategory': 'Edible Oils & Ghee'}, {'name': 'Mundial Olive Oil Pomace Tin 4 Liter', 'price': 4556, 'category': 'Foodgrains, Oil & Masala', 'subCategory': 'Edible Oils & Ghee'}, {'name': 'Habib Cooking Oil 2.5Ltr Tin', 'price': 711, 'category': 'Foodgrains, Oil & Masala', 'subCategory': 'Edible Oils & Ghee'}, {'name': 'Aseel Ghee Butter Tin 400g', 'pr

26: [{'name': 'Shan Special Shahi Haleem 300grams', 'price': 149, 'category': 'Foodgrains, Oil & Masala', 'subCategory': 'Recipe Masala'}, {'name': 'Shan Murgh Cholay 50grams', 'price': 80, 'category': 'Foodgrains, Oil & Masala', 'subCategory': 'Recipe Masala'}, {'name': 'Shan Sindhi Biryani 60grams', 'price': 80, 'category': 'Foodgrains, Oil & Masala', 'subCategory': 'Recipe Masala'}, {'name': 'National Recipes Bombay Biryani Economy Pack 140g', 'price': 123, 'category': 'Foodgrains, Oil & Masala', 'subCategory': 'Recipe Masala'}, {'name': 'Shan Chicken Handi 50grams', 'price': 80, 'category': 'Foodgrains, Oil & Masala', 'subCategory': 'Recipe Masala'}, {'name': 'Shan Chicken Tikka 50grams', 'price': 80, 'category': 'Foodgrains, Oil & Masala', 'subCategory': 'Recipe Masala'}, {'name': 'Shan Keema 50grams', 'price': 69, 'category': 'Foodgrains, Oil & Masala', 'subCategory': 'Recipe Masala'}, {'name': 'National Recipes Chapli Kabab 100g', 'price': 123, 'category': 'Foodgrains, Oil & Mas

...

213: [{'name': 'Chukandar - Beetroot - 250 grams', 'price': 53, 'category': 'Vegetables', 'subCategory': 'Fresh Vegetables'}, {'name': 'lemon - Nimbo 250 grams', 'price': 131, 'category': 'Vegetables', 'subCategory': 'Fresh Vegetables'}, {'name': 'Lauki kaddu Bottle Gourd 500 grams', 'price': 78, 'category': 'Vegetables', 'subCategory': 'Fresh Vegetables'}, {'name': 'Bhindi - Ladies Finger - 1 kg', 'price': 267, 'category': 'Vegetables', 'subCategory': 'Fresh Vegetables'}, {'name': 'Karela Bitter Melon 500 grams', 'price': 86, 'category': 'Vegetables', 'subCategory': 'Fresh Vegetables'}, {'name': 'Iceberg Lettuce - Salad 100 grams', 'price': 159, 'category': 'Vegetables', 'subCategory': 'Fresh Vegetables'}, {'name': 'Kakri Cucumis 500 grams', 'price': 50, 'category': 'Vegetables', 'subCategory': 'Fresh Vegetables'}, {'name': 'Turai - Tori - luffa 500 grams', 'price': 53, 'category': 'Vegetables', 'subCategory': 'Fresh Vegetables'}, {'name': 'Curry Patta Gaddi - Kari patta Bunch', 'pric

220: [{'name': 'Veet Cream - Silk and Fresh 100g (Dry Skin)', 'price': 408, 'category': 'Hygiene', 'subCategory': 'Feminine Hygiene'}, {'name': 'Veet Cream - Silk and Fresh 100g (Normal Skin)', 'price': 389, 'category': 'Hygiene', 'subCategory': 'Feminine Hygiene'}, {'name': 'Veet Cream - Silk and Fresh 25g (Dry Skin)', 'price': 113, 'category': 'Hygiene', 'subCategory': 'Feminine Hygiene'}, {'name': 'Veet Cream - Silk and Fresh 25g (Normal Skin)', 'price': 113, 'category': 'Hygiene', 'subCategory': 'Feminine Hygiene'}, {'name': 'Veet Cream - Brightening 25g (Normal to Dry Skin)', 'price': 98, 'category': 'Hygiene', 'subCategory': 'Feminine Hygiene'}, {'name': "EU Wax Strips Lavender 10's", 'price': 153, 'category': 'Hygiene', 'subCategory': 'Feminine Hygiene'}, {'name': 'EU Wax Strips Chocolate 10s', 'price': 153, 'category': 'Hygiene', 'subCategory': 'Feminine Hygiene'}, {'name': 'Veet Cream - Brightening 50g (Normal to Dry Skin)', 'price': 210, 'category': 'Hygiene', 'subCategory': 

227: [{'name': "Young's Mayonnaise 1kg Pouch", 'price': 301, 'category': 'Spread & Chutney', 'subCategory': 'Spreads & Sauces'}, {'name': "Young's Chicken Spread 300ml", 'price': 238, 'category': 'Spread & Chutney', 'subCategory': 'Spreads & Sauces'}, {'name': "Young's Chicken Bar-B-Que Spread 200ml", 'price': 164, 'category': 'Spread & Chutney', 'subCategory': 'Spreads & Sauces'}, {'name': "Young's Chicken Spread 500ml", 'price': 370, 'category': 'Spread & Chutney', 'subCategory': 'Spreads & Sauces'}, {'name': "Young's Mayonnaise 200ml", 'price': 82, 'category': 'Spread & Chutney', 'subCategory': 'Spreads & Sauces'}, {'name': "Young's Mayo Garlic 500ml", 'price': 270, 'category': 'Spread & Chutney', 'subCategory': 'Spreads & Sauces'}, {'name': "Young's Mayo Garlic 200ml", 'price': 122, 'category': 'Spread & Chutney', 'subCategory': 'Spreads & Sauces'}, {'name': 'Shangrila Mango Chilli Sauce, 370g', 'price': 238, 'category': 'Spread & Chutney', 'subCategory': 'Spreads & Sauces'}, {'nam

234: [{'name': 'Shangrila Ketchup 1kg', 'price': 232, 'category': 'Spread & Chutney', 'subCategory': 'Ketchup'}]
235: [{'name': 'National Spicy Mixed Pickle, 310g', 'price': 138, 'category': 'Spread & Chutney', 'subCategory': 'Pickles & Chutney'}, {'name': 'Ahmed Mango Pickle 330gm', 'price': 132, 'category': 'Spread & Chutney', 'subCategory': 'Pickles & Chutney'}, {'name': 'National Kasundi Mango Pickle, 320g', 'price': 138, 'category': 'Spread & Chutney', 'subCategory': 'Pickles & Chutney'}, {'name': "Mitchell's Plum Chutney 420g", 'price': 296, 'category': 'Spread & Chutney', 'subCategory': 'Pickles & Chutney'}, {'name': "Mitchell's Mango Chutney 420g", 'price': 296, 'category': 'Spread & Chutney', 'subCategory': 'Pickles & Chutney'}, {'name': "Mitchell's Garlic Pickle 360g", 'price': 211, 'category': 'Spread & Chutney', 'subCategory': 'Pickles & Chutney'}, {'name': 'Ahmed Carrot In Sugar Syrup 450gm', 'price': 169, 'category': 'Spread & Chutney', 'subCategory': 'Pickles & Chutney'}

[{'name': 'Maida 500 grams',
  'price': 48,
  'category': 'Foodgrains, Oil & Masala',
  'subCategory': 'Atta, Flours & Sooji'},
 {'name': 'Sooji Semolina - Suji Powder - 500 grams',
  'price': 39,
  'category': 'Foodgrains, Oil & Masala',
  'subCategory': 'Atta, Flours & Sooji'},
 {'name': 'Baisan - Besan 500 grams',
  'price': 95,
  'category': 'Foodgrains, Oil & Masala',
  'subCategory': 'Atta, Flours & Sooji'},
 {'name': 'Mughal Chakki Atta Flour - 5 Kg',
  'price': 370,
  'category': 'Foodgrains, Oil & Masala',
  'subCategory': 'Atta, Flours & Sooji'},
 {'name': 'Sunridge Chakki Atta Flour - 5kg',
  'price': 475,
  'category': 'Foodgrains, Oil & Masala',
  'subCategory': 'Atta, Flours & Sooji'},
 {'name': 'Sunridge Chakki Atta Flour - 10kg',
  'price': 824,
  'category': 'Foodgrains, Oil & Masala',
  'subCategory': 'Atta, Flours & Sooji'},
 {'name': 'Ashrafi Fine Flour - Atta Poly Bag - 5kg',
  'price': 337,
  'category': 'Foodgrains, Oil & Masala',
  'subCategory': 'Atta, Flours &

In [12]:
df = pd.DataFrame(d)

df.to_excel("grocer.pk_assortment.xlsx", index=False)